# **Déploiement de l'API**

Mlflow ayant été imposé pour le *tracking* d'expérience, nous utiliserons la même solution pour le *serving* de modèle.

Pour servir un modèle :

```sh
$py3105 mlflow models serve -m my_model
```

Signature du modèle

In [ ]:
from mlflow.models.signature import infer_signature

signature = infer_signature(X_train, y_train)

Sauvegarde du modèle : elle a déjà été faite par log_model au moment de mener les expériences d'entraînement et de validation.

En revanche, nous avons commis l'erreur de ne pas intégrer les derniers traitements de scaling et d'imputation dans une pipeline (un objet de type classifieur qui puisse être passé en argument à l'enregistrement de modèle mlflow).

Cela signifie que le client devra réaliser cette partie du traitement.

In [ ]:
import mlflow
mlflow.sklearn.save_model(pipeline, 'mlflow_model', signature=signature)

Mise en service :

```sh
mlflow models serve -m mlflow_model/
```

Requêtage :

```sh
curl http://127.0.0.1:5000/invocations -H 'Content-Type: application/json' -d '{"data": [[1, 2, 3, 4, 5, 6, 7, 8]]}'
```

# Flask

https://flask.palletsprojects.com/en/2.3.x/quickstart/

Application minimale :

```python
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello_world():
    return "<p>Hello, World!</p>"
```

Mise en service:
```sh
py -m flask --app .\modules\home_credit\flask_app_1_hello run
```

```
 * Serving Flask app '.\modules\home_credit\flask_api'
 * Debug mode: off
WARNING: This is a development server. Do not use it in a production deployment. Use a **production WSGI server** instead.
 * Running on http://127.0.0.1:5000
 ```

Ouverture publique (par défaut mode privé de développement) :

```sh
$ flask run --host=0.0.0.0
```

Exécution en mode debug (JAMAIS EN PRODUCTION!!!!):

```sh
$ flask --app hello run --debug
```

Se protéger contre l'attaque par injection de code : `escape`

```python
from markupsafe import escape

@app.route("/<name>")
def hello(name):
    return f"Hello, {escape(name)}!"
```

à tester avec et sans escape et avec la valeur de `name`: `<script>alert("bad")</script>`

J'ai tenté avec http://127.0.0.1:5000/%3Cscript%3Ealert(%22bad%22)%3C/script%3E, mais ça ne fonctionne pas !


In [2]:
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello_world():
    return "<p>Hello, World!</p>"

In [ ]:
from flask import Flask
from markupsafe import escape

app = Flask(__name__)

@app.route("/<name>")
def hello(name):
    return f"Hello, {escape(name)}!"

|type|desc|
|-|-|
|string|(default) accepts any text without a slash|
|int|accepts positive integers|
|float|accepts positive floating point values|
|path|like string but also accepts slashes|
|uuid|accepts UUID strings|

In [ ]:
from flask import Flask
from markupsafe import escape

app = Flask(__name__)

@app.route('/user/<username>')
def show_user_profile(username):
    # show the user profile for that user
    return f'User {escape(username)}'

@app.route('/post/<int:post_id>')
def show_post(post_id):
    # show the post with the given id, the id is an integer
    return f'Post {post_id}'

@app.route('/path/<path:subpath>')
def show_subpath(subpath):
    # show the subpath after /path/
    return f'Subpath {escape(subpath)}'

URL Building

# Déboguer en ligne le Flask, juste pas confortable

In [1]:
from flask import Flask, jsonify
from home_credit.load import load_prep_dataset
from home_credit.persist import load_model
from home_credit.best_model_search import train_preproc, predict
from sklearn.preprocessing import MinMaxScaler

# Chargement du jeu de données
data = load_prep_dataset("baseline_v1")
print(f"baseline_v1 loaded {data.shape}")

# Charger le classifieur
model = load_model("lgbm_baseline_default_third_party")
print(f"classifier loaded {model}")

baseline_v1 loaded (356255, 790)
classifier loaded LGBMClassifier(callbacks=None, colsample_bytree=0.9497036, learning_rate=0.02,
               max_depth=8, min_child_weight=39.3259775,
               min_split_gain=0.0222415, n_estimators=10000, n_jobs=4,
               num_leaves=34, reg_alpha=0.041545473, reg_lambda=0.0735294,
               subsample=0.8715623)


C:\Users\franc\Projects\pepper_credit_scoring_tool\modules\home_credit\persist.py:193: UserWarning: Using a non-skops.io save engine can potentially load malicious objects. It is recommended to use skops.io instead.
  warnings.warn(


Bon, l'erreur de conception de ne pas intégrer en pipeline le scaling et l'imputing me vaut ici de ne pas pouvoir terminer à temps : il faut tout dégrapher et reprendre en arrière, plusieurs jours de travail.

Donc je livre mon api en l'état, et il me maquera le dash streamlit et le déploiement Azur.

In [2]:
def customer(customer_id):
    print("customer id:", customer_id)
    customer = data[data.SK_ID_CURR == customer_id]
    display(customer)
    x, y_true = train_preproc(customer, MinMaxScaler(), keep_test_samples=True)
    print("y_true:", y_true)
    y_prob = model.predict_proba(x)[:, 1]
    print("y_prob:", y_prob)
    """y_pred = int(y_prob > .4)
    print("y_pred:", y_pred)
    return jsonify({
        "y_true": y_true,
        "y_prob": y_prob,
        "y_pred": y_pred
    })"""

In [3]:
customer(100002)

customer id: 100002


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,CC_NAME_CONTRACT_STATUS_Sent_proposal_MIN_nan,CC_NAME_CONTRACT_STATUS_Sent_proposal_MAX_False,CC_NAME_CONTRACT_STATUS_Sent_proposal_MAX_True,CC_NAME_CONTRACT_STATUS_Sent_proposal_MAX_nan,CC_NAME_CONTRACT_STATUS_Signed_MIN_False,CC_NAME_CONTRACT_STATUS_Signed_MIN_True,CC_NAME_CONTRACT_STATUS_Signed_MIN_nan,CC_NAME_CONTRACT_STATUS_Signed_MAX_False,CC_NAME_CONTRACT_STATUS_Signed_MAX_True,CC_NAME_CONTRACT_STATUS_Signed_MAX_nan
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,...,True,False,False,True,False,False,True,False,False,True


DBG data_train.shape: (1, 790)
DBG default_imputation data.shape: (1, 790)
DBG default_imputation data_train.shape: (1, 790)
DBG default_imputation new_data.shape: (1, 635)


ValueError: Shape of passed values is (1, 635), indices imply (1, 790)